# TrajectoryExplorer Demo

In [ ]:
import math
import matplotlib.pyplot as plt

from kbmod.trajectory_explorer import TrajectoryExplorer

## TrajectoryExplorer

`TrajectoryExplorer` is a class that allows us to evaluate individual trajectories for debugging or scientific analysis. It handles all the loading of data onto GPU, configuration of the parameters, and other set up. 

### Create fake data
Let's start by setting up a fake data set with a single known object.

In [ ]:
from kbmod.fake_data.fake_data_creator import create_fake_times, FakeDataSet
from kbmod.trajectory_utils import make_trajectory

# Create a list of fake times from MJD=57130.2 onward
num_images = 12
fake_times = create_fake_times(num_images, t0=57130.2)

# Create fake data set.
fake_ds = FakeDataSet(
    400,  # Width
    500,  # Height
    fake_times,
    noise_level=2.0,
    psf_val=1.0,
)

# Create a bright fake object starting at pixel x=50, y=60 and moving with velocity vx=5.0, vy=-2.0.
fake_trj = make_trajectory(50, 60, 5.0, -2.0, flux=100.0)
fake_ds.insert_object(fake_trj)

# Display the image from the first time step.
img = fake_ds.stack.get_single_image(0).get_science().image
plt.imshow(img, cmap="gray")

To make things more interesting, let's block out the pixels for timestep 6, so we cannot see the object then. We need to do this manually (there is no helper function to do this).

In [ ]:
# Remove at least observation from the trajectory.
dt = fake_times[6] - fake_times[0]
pred_x = int(50 + 5.0 * dt)
pred_y = int(60 - 2.0 * dt)

sci_t6 = fake_ds.stack.get_single_image(6).get_science()
for dy in [-2, -1, 0, 1, 2]:
    for dx in [-2, -1, 0, 1, 2]:
        sci_t6.set_pixel(pred_y + dy, pred_x + dx, 0.00001)

img = fake_ds.stack.get_single_image(6).get_science().image
plt.imshow(img, cmap="gray")

## Using TrajectoryExplorer

`TrajectoryExplorer` is constructed from the `ImageStack` and an optional configuration. Once it is instantiated, we can use the `evaluate_linear_trajectory` function to query specific trajectories. The function returns a `ResultRow` with a variety of information. The most important is a `Trajectory` object that stores the raw statistics coming from the GPU search.

In [ ]:
explorer = TrajectoryExplorer(fake_ds.stack)

result = explorer.evaluate_linear_trajectory(50, 60, 5.0, -2.0)
print(result.trajectory)

We also have the row psi-curves, phi-curves, and likelihood curves:

In [ ]:
fig, axs = plt.subplots(3, 1)
axs[0].plot(fake_times, result.psi_curve)
axs[0].set_title("Psi")

axs[1].plot(fake_times, result.phi_curve)
axs[1].set_title("Psi")

axs[2].plot(fake_times, result.likelihood_curve)
axs[2].set_title("Likelihood")

We can also extract the stamps for those time steps.

In [ ]:
w = 4
h = math.ceil(num_images / w)

fig, axs = plt.subplots(h, w)
for i in range(h):
    for j in range(w):
        ind = w * i + j
        if ind < num_images:
            axs[i, j].imshow(result.all_stamps[ind], cmap="gray")

## Using (RA, dec)

You can also use the `TrajectoryExplorer` with a WCS and (RA, dec) coordinates. Both RA and dec are specified in degrees and the corresponding velocities are expressed as degrees per day.

We start by creating a fake WCS to match our fake images. Then we evaluate the trajectory based on this WCS.

In [ ]:
from astropy.wcs import WCS

my_wcs = WCS(naxis=2)
my_wcs.wcs.crpix = [201.0, 251.0]  # Reference point on the image (center of the image 1-indexed)
my_wcs.wcs.crval = [45.0, -15.0]  # Reference pointing on the sky
my_wcs.wcs.cdelt = [0.1, 0.1]  # Pixel step size
my_wcs.wcs.ctype = ["RA---TAN-SIP", "DEC--TAN-SIP"]

# Use the WCS to compute the angular coordinates of the inserted object at two times.
sky_pos0 = my_wcs.pixel_to_world(50, 60)
sky_pos1 = my_wcs.pixel_to_world(55, 58)

ra0 = sky_pos0.ra.deg
dec0 = sky_pos0.dec.deg
v_ra = sky_pos1.ra.deg - ra0
v_dec = sky_pos1.dec.deg - dec0
print(f"Object starts at ({ra0}, {dec0}) with velocity ({v_ra}, {v_dec}).")

result = explorer.evaluate_angle_trajectory(ra0, dec0, v_ra, v_dec, my_wcs)
print(result.trajectory)

## Filtering

A key component of KBMOD is the clipped Sigma-G filtering. `TrajectoryExplorer` does not do any filtering be default so that users can see all the information for the given trajectory. However, it provides the ability to apply the filtering manually. The unfiltered time stamps are stored in `valid_indices`.

In [ ]:
explorer.apply_sigma_g(result)
print(f"Valid time steps={result.valid_indices}")